In [1]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [2]:
from gensim import corpora, models, similarities

In [3]:
ls

README.md                                   gensim LDA on papers for different K.ipynb  paper_titles_and_abstracts.csv              venv/


In [8]:
import csv
with open('Papers.csv', 'rb') as f:
    data = [row for row in csv.reader(f.read().splitlines())]
#print data

In [11]:
limit = 3
l = 0
for d in data:
    print d
    l += 1
    if l > limit: break

['Id', 'Title', 'EventType', 'PdfName', 'Abstract', 'PaperText']
['5677', 'Double or Nothing: Multiplicative Incentive Mechanisms for Crowdsourcing', 'Poster', '5677-double-or-nothing-multiplicative-incentive-mechanisms-for-crowdsourcing.pdf', 'Crowdsourcing has gained immense popularity in machine learning applications for obtaining large amounts of labeled data. Crowdsourcing is cheap and fast, but suffers from the problem of low-quality data. To address this fundamental challenge in crowdsourcing, we propose a simple payment mechanism to incentivize workers to answer only the questions that they are sure of and skip the rest. We show that surprisingly, under a mild and natural no-free-lunch requirement, this mechanism is the one and only incentive-compatible payment mechanism possible. We also show that among all possible incentive-compatible  mechanisms (that may or may not satisfy no-free-lunch), our mechanism makes the smallest possible payment to spammers.  Interestingly, this u